## Практическое задание к уроку 7 по теме "Рекуррентные сети для обработки последовательностей".

1. Обучите нейронную сеть GRU/LSTM для предсказания сентимента сообщений с твитера на примере https://www.kaggle.com/datasets/arkhoshghalb/twitter-sentiment-analysis-hatred-speech

2. Опишите, какой результат вы получили? Что помогло вам улучшить ее точность?

Данная задача во многом повторяет предыдущее практическое задание, поэтому  
скопируем оттуда часть, отвечающую за подготовку данных.

Загрузим необходимые библиотеки и данные:

In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from string import punctuation
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchinfo import summary
from tqdm import tqdm

In [2]:
RANDOM_STATE = 29

In [3]:
df_train = pd.read_csv('../Lesson_6/data/train.csv', index_col='id')
print(df_train.shape)
df_train.head()

(31962, 2)


,label,tweet
id,,
1,0,@user when a father is dysfunctional and is s...
2,0,@user @user thanks for #lyft credit i can't us...
3,0,bihday your majesty
4,0,#model i love u take with u all the time in ...
5,0,factsguide: society now #motivation


<ins>Описание датасета:</ins>  
The objective of this task is to detect hate speech in tweets.  
For the sake of simplicity, we say a tweet contains hate speech  
if it has a racist or sexist sentiment associated with it.  
So, the task is to classify racist or sexist tweets from other tweets.  
  
Formally, given a training sample of tweets and labels, where label '1'  
denotes the tweet is racist/sexist and label '0' denotes the tweet is  
not racist/sexist, your objective is to predict the labels on the test dataset.

Таким образом, нам нужно будет искать твиты, которые содержат  
расистский или сексистский смысл.

In [4]:
df_test = pd.read_csv('../Lesson_6/data/test.csv', index_col='id')
print(df_test.shape)
df_test.head()

(17197, 1)


,tweet
id,
31963,#studiolife #aislife #requires #passion #dedic...
31964,@user #white #supremacists want everyone to s...
31965,safe ways to heal your #acne!! #altwaystohe...
31966,is the hp and the cursed child book up for res...
31967,"3rd #bihday to my amazing, hilarious #nephew..."


Так как тестовые данные не содержат меток, то будем использовать только  
трейн для обучения и валидации, чтобы можно было оценить качество модели.  
Посмотрим на баланс классов:

In [5]:
df_train['label'].value_counts()

0    29720
1     2242
Name: label, dtype: int64

In [6]:
df_train['label'].value_counts()[0] / df_train['label'].value_counts()[1]

13.256021409455842

Как часто бывает в подобных задачах, мы имеем большой дисбаланс классов.

Сделаем разбивку на трейн и валидацию:

In [7]:
df_train, df_val = train_test_split(df_train, 
                                    test_size=0.2, 
                                    random_state=RANDOM_STATE, 
                                    stratify=df_train['label'])

df_train.shape, df_val.shape

((25569, 2), (6393, 2))

Сделаем подготовку текстов:

In [8]:
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /home/shkin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/shkin/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [9]:
puncts = set(punctuation)
# Не будем очищать текст от апострофов, заменим их потом на пробелы,
# т.к. встроенные в nltk английские стопслова и так потом отфильтруют лишнее
puncts = puncts - {"'"}

In [10]:
def preprocess_text(txt):
    txt = str(txt)
    txt = ''.join(char for char in txt if char not in puncts) # очистка от пунктуации
    txt = txt.replace("'", " ")
    txt = txt.lower().split()
    txt = [word for word in txt if word.isalpha()] # очистка от символов и цифр
    txt = [lemmatizer.lemmatize(word) for word in txt] # лемматизация
    txt = [word for word in txt if word not in stopwords.words('english')] # очистка от стопслов
    return ' '.join(txt)

In [11]:
tqdm.pandas()

df_train['tweet'] = df_train['tweet'].progress_apply(preprocess_text)
df_val['tweet'] = df_val['tweet'].progress_apply(preprocess_text)

100%|█████████████████████████████████████| 6393/6393 [00:04<00:00, 1317.09it/s]


In [12]:
df_train.head()

,label,tweet
id,,
14553,0,user amazing wait see going cantwait
2563,0,wait new user trailer gamer
12125,0,thriving iam positive affirmation
6326,0,happy new user book lil upset page faded user ...
3996,0,arrive cold rainy english noh first time back ...


Подготовим общий корпус текста:

In [13]:
train_corpus = ''.join(df_train['tweet'].values)

Сделаем токенизацию:

In [14]:
tokens = word_tokenize(train_corpus)
tokens[:5]

['user', 'amazing', 'wait', 'see', 'going']

Создадим словарь:

In [15]:
MAX_WORDS = 4000
MAX_LEN = 40

In [16]:
dist = FreqDist(tokens)
tokens_top = [items[0] for items in dist.most_common(MAX_WORDS - 1)]

In [17]:
tokens_top[:10]

['user', 'day', 'love', 'u', 'amp', 'like', 'life', 'happy', 'get', 'wa']

In [18]:
vocabulary = {word: count for count, word in dict(enumerate(tokens_top, 1)).items()}

Переведём твиты в набор индексов, добавим паддинг:

In [19]:
def text_to_sequence(txt, maxlen):
    result = []
    tokens = word_tokenize(txt)
    for word in tokens:
        if word in vocabulary:
            result.append(vocabulary[word])

    padding = [0] * (maxlen-len(result))
    return result[-maxlen:] + padding

In [20]:
X_train = np.array([text_to_sequence(txt, MAX_LEN) for txt in df_train['tweet'].values])
X_val = np.array([text_to_sequence(txt, MAX_LEN) for txt in df_val['tweet'].values])

X_train.shape, X_val.shape

((25569, 40), (6393, 40))

In [21]:
print(f"Оригинальная строка: {df_train['tweet'].iloc[5]}")
print(f"Обработанная строка: {X_train[5]}")

Оригинальная строка: found beautiful one bedroom double stall garage patio amp huge kitchen signed lease wait move
Обработанная строка: [ 172   51   19 1233 3015 3475    5  777 1537 1538   68  694    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]


Инициализируем рекуррентную нейросеть:

In [22]:
class Net(nn.Module):
    def __init__(self, vocab_size=2000, embedding_dim=128, out_dim=64, use_last=True, threshold=0.5, num_classes=1):
        super().__init__()
        self.threshold = threshold
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0) 
        self.gru = nn.GRU(embedding_dim, out_dim, batch_first=True) 
        self.linear = nn.Linear(out_dim, num_classes)
        self.dp = nn.Dropout(0.5)
        self.use_last = use_last
        
    def forward(self, x):                          
        x = self.embedding(x)
        x = self.dp(x)
        x, _ = self.gru(x)
           
        if self.use_last:
            x = x[:,-1,:]
        else:
            x = torch.mean(x[:,:], dim=1)
            
        x = self.dp(x)
        x = self.linear(x)
        x = torch.sigmoid(x)
        return x
    
    def predict(self, x):
        x = torch.IntTensor(x).to(device)
        x = self.forward(x)
        x = torch.squeeze((x > self.threshold).int())
        return x

Посмотрим структуру сети:

In [23]:
summary(Net(), input_data=torch.IntTensor(X_train[np.newaxis, 0]))

Layer (type:depth-idx)                   Output Shape              Param #
Net                                      [1, 1]                    --
├─Embedding: 1-1                         [1, 40, 128]              256,000
├─Dropout: 1-2                           [1, 40, 128]              --
├─GRU: 1-3                               [1, 40, 64]               37,248
├─Dropout: 1-4                           [1, 64]                   --
├─Linear: 1-5                            [1, 1]                    65
Total params: 293,313
Trainable params: 293,313
Non-trainable params: 0
Total mult-adds (M): 1.75
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 1.17
Estimated Total Size (MB): 1.23

Подготовим датасеты:

In [24]:
class DataWrapper(Dataset):
    def __init__(self, data, target):

        self.data = torch.from_numpy(data)
        self.target = torch.from_numpy(target)
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
            
        return x, y
    
    def __len__(self):
        return self.data.shape[0]

In [25]:
BATCH_SIZE = 512

In [26]:
torch.random.manual_seed(RANDOM_STATE)

train_dataset = DataWrapper(X_train, df_train['label'].values)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

val_dataset = DataWrapper(X_val, df_val['label'].values)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [27]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

Напишем код сети. Учитывая дисбаланс классов, метрика accuracy нам  
не подходит. Вместо неё будем использовать F1-score.

In [28]:
def train_nn(epochs=5, embedding_dim=128, hidden_size=64, lr=1e-3, threshold=0.5, use_last=True, return_model=False):

    torch.random.manual_seed(RANDOM_STATE)
    torch.backends.cudnn.deterministic = True

    net = Net(vocab_size=MAX_WORDS, embedding_dim=embedding_dim, 
              out_dim=hidden_size, use_last=use_last, threshold=threshold).to(device)
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.BCELoss()

    for epoch in range(epochs):
        train_losses = np.array([])
        test_losses = np.array([])
        tp, fp, tn, fn = 0, 0, 0, 0

        for i, (inputs, labels) in enumerate(train_loader):
            net.train()
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = net(inputs)

            loss = criterion(outputs, labels.float().view(-1, 1))
            loss.backward()
            optimizer.step()

            train_losses = np.append(train_losses, loss.item())

            net.eval()
            outputs = torch.squeeze((net(inputs) > threshold).int())

            tp += ((labels == 1) & (outputs == 1)).sum().item()
            tn += ((labels == 0) & (outputs == 0)).sum().item()
            fp += ((labels == 0) & (outputs == 1)).sum().item()
            fn += ((labels == 1) & (outputs == 0)).sum().item()

        precision = tp / (tp + fp) if (tp + fp) != 0 else 0
        recall = tp / (tp + fn) if (tp + fn) != 0 else 0

        f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) != 0 else 0

        print(f'Epoch [{epoch + 1}/{epochs}]. ' \
              f'Loss: {train_losses.mean():.3f}. ' \
              f'F1-score: {f1_score:.3f}', end='. ')

        tp, fp, tn, fn = 0, 0, 0, 0

        with torch.no_grad():
            for i, (inputs, labels) in enumerate(val_loader):

                inputs, labels = inputs.to(device), labels.to(device)
                outputs = net(inputs)

                loss = criterion(outputs, labels.float().view(-1, 1))
                test_losses = np.append(test_losses, loss.item())

                tp += ((labels == 1) & (torch.squeeze((outputs > threshold).int()) == 1)).sum()
                tn += ((labels == 0) & (torch.squeeze((outputs > threshold).int()) == 0)).sum()
                fp += ((labels == 0) & (torch.squeeze((outputs > threshold).int()) == 1)).sum()
                fn += ((labels == 1) & (torch.squeeze((outputs > threshold).int()) == 0)).sum()

        precision = tp / (tp + fp) if (tp + fp) != 0 else 0
        recall = tp / (tp + fn) if (tp + fn) != 0 else 0

        f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) != 0 else 0

        print(f'Test loss: {test_losses.mean():.3f}. Test F1-score: {f1_score:.3f}. Precision: {precision:.3f}. Recall: {recall:.3f}')

    print('Training is finished!')
    if return_model:
        return net

Обучим модель на 70 эпохах:

In [29]:
train_nn(epochs=70, use_last=True)

Epoch [1/70]. Loss: 0.456. F1-score: 0.098. Test loss: 0.265. Test F1-score: 0.000. Precision: 0.000. Recall: 0.000
Epoch [2/70]. Loss: 0.263. F1-score: 0.000. Test loss: 0.252. Test F1-score: 0.000. Precision: 0.000. Recall: 0.000
Epoch [3/70]. Loss: 0.260. F1-score: 0.000. Test loss: 0.247. Test F1-score: 0.000. Precision: 0.000. Recall: 0.000
Epoch [4/70]. Loss: 0.250. F1-score: 0.000. Test loss: 0.228. Test F1-score: 0.000. Precision: 0.000. Recall: 0.000
Epoch [5/70]. Loss: 0.219. F1-score: 0.000. Test loss: 0.187. Test F1-score: 0.000. Precision: 0.000. Recall: 0.000
Epoch [6/70]. Loss: 0.192. F1-score: 0.040. Test loss: 0.170. Test F1-score: 0.081. Precision: 1.000. Recall: 0.042
Epoch [7/70]. Loss: 0.176. F1-score: 0.377. Test loss: 0.156. Test F1-score: 0.498. Precision: 0.767. Recall: 0.368
Epoch [8/70]. Loss: 0.164. F1-score: 0.559. Test loss: 0.149. Test F1-score: 0.545. Precision: 0.716. Recall: 0.440
Epoch [9/70]. Loss: 0.152. F1-score: 0.636. Test loss: 0.147. Test F1-sc

Модель GRU показала результат лучше, чем 1D свёртки из прошлого практического  
задания, F1-score выше где-то на 8%. Также она оказалась чуть лучше, чем модель  
LSTM (выявлено в результате перебора гиперпараметров). По логам видно, что  
переобучение снова присутствует. Интересно, что модель GRU положительно отреагировала  
на увеличение размера словаря и длины последовательности, тогда как свёрточная сеть  
никак на это не реагировала.  
Финальную модель обучим на 30 эпохах, где у нас относительно малый тест лосс  
и относительно высокая метрика. Так же, как и в прошлый раз, снизим порог  
классификации для получения более высокого Recall, который важен в нашей задаче:

In [30]:
my_net = train_nn(epochs=30, threshold=0.25, return_model=True)

Epoch [1/30]. Loss: 0.456. F1-score: 0.124. Test loss: 0.265. Test F1-score: 0.000. Precision: 0.000. Recall: 0.000
Epoch [2/30]. Loss: 0.263. F1-score: 0.000. Test loss: 0.252. Test F1-score: 0.000. Precision: 0.000. Recall: 0.000
Epoch [3/30]. Loss: 0.260. F1-score: 0.000. Test loss: 0.247. Test F1-score: 0.000. Precision: 0.000. Recall: 0.000
Epoch [4/30]. Loss: 0.250. F1-score: 0.000. Test loss: 0.228. Test F1-score: 0.000. Precision: 0.000. Recall: 0.000
Epoch [5/30]. Loss: 0.219. F1-score: 0.347. Test loss: 0.187. Test F1-score: 0.453. Precision: 0.505. Recall: 0.411
Epoch [6/30]. Loss: 0.192. F1-score: 0.534. Test loss: 0.170. Test F1-score: 0.508. Precision: 0.552. Recall: 0.471
Epoch [7/30]. Loss: 0.176. F1-score: 0.602. Test loss: 0.156. Test F1-score: 0.556. Precision: 0.544. Recall: 0.569
Epoch [8/30]. Loss: 0.164. F1-score: 0.643. Test loss: 0.149. Test F1-score: 0.579. Precision: 0.589. Recall: 0.569
Epoch [9/30]. Loss: 0.152. F1-score: 0.672. Test loss: 0.147. Test F1-sc

По сравнению с предыдущим решением на свёртках, мы находим почти столько же  
оскорбительных твитов (64-65%). Но показатель точности теперь выше на 21%  
(65% против 44%). В общем, эти результаты всё равно являются не очень хорошими для  
готовой модели. Снова будем считать, что основной причиной является недостаток данных  
(25 тысяч примеров в обучающей выборке, из которых всего 1800 положительного  
класса).

In [31]:
df_train['label'].value_counts()

0    23775
1     1794
Name: label, dtype: int64

Предсказание модели:

In [32]:
my_net.predict(X_val[np.newaxis, 0])

tensor(1, device='cuda:0', dtype=torch.int32)

Задача выполнена.